In [1]:
from pydisagg.disaggregate import split_datapoint
import numpy as np
from pydisagg.models import RateMultiplicativeModel
from pydisagg.models import LMOModel
from pydisagg.models import LogOddsModel
import pandas as pd
from tqdm.auto import tqdm

In [2]:
populations=np.array([20,10,5])
rate_pattern=np.array([0.1,0.5,0.7])
observed_total=20.
observed_total_SE=0
pattern_cov = 0.01*np.identity(3)
zero_cov = np.zeros((3,3))

In [3]:
oddm=LogOddsModel()

In [6]:
b=oddm.fit_beta(observed_total,rate_pattern,populations)

C=oddm.count_split_covariance_uncertainty(b,rate_pattern,populations,observed_total_SE,pattern_cov)
np.around(C,3)


array([[ 2.187, -1.762, -0.425],
       [-1.762,  1.452,  0.31 ],
       [-0.425,  0.31 ,  0.115]])

In [9]:
oddm.split_to_counts(observed_total,rate_pattern,populations)

array([7.08400681, 8.31542362, 4.60056957])

In [5]:
C=oddm.count_split_covariance_uncertainty(b,rate_pattern,populations,2,zero_cov)
C

array([[2.0806605 , 0.63708815, 0.16715031],
       [0.63708815, 0.1950733 , 0.05118061],
       [0.16715031, 0.05118061, 0.01342806]])

In [10]:
C=oddm.rate_split_covariance_uncertainty(b,rate_pattern,populations,2,zero_cov)
C

array([[0.00520165, 0.00318544, 0.0016715 ],
       [0.00318544, 0.00195073, 0.00102361],
       [0.0016715 , 0.00102361, 0.00053712]])

In [12]:
np.diag(populations)@C@np.diag(populations)

array([[2.0806605 , 0.63708815, 0.16715031],
       [0.63708815, 0.1950733 , 0.05118061],
       [0.16715031, 0.05118061, 0.01342806]])

In [18]:
populations=np.array([20,10,5])
rate_pattern=np.array([0.1,0.5,0.7])
observed_total=20.
variance_scale=0.0001
pattern_cov = variance_scale*np.identity(3)
zero_cov = np.zeros((3,3))

num_samples = 1000
pattern_draws = np.random.multivariate_normal(rate_pattern,pattern_cov,int(1.5*num_samples))
pattern_draws = pattern_draws[np.min(pattern_draws,axis=1)>0.01][:num_samples]
total_draws = observed_total + np.sqrt(variance_scale)*np.random.randn(num_samples)

In [19]:
splits = [
    oddm.split_to_rates(tot,pattern,populations) for tot,pattern in tqdm(zip(total_draws,pattern_draws),
                                                                         total=num_samples)
                                                                         ]
splits = np.array(splits)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [20]:
draw_cov = np.cov(splits.T)
asymptotic_cov = oddm.rate_split_covariance_uncertainty(
    oddm.fit_beta(observed_total,rate_pattern,populations),
    rate_pattern,
    populations,
    np.sqrt(variance_scale),
    pattern_cov
    )
print("Estimated covariance matrix from draws")
print(draw_cov)

print("Estimated covariance matrix from delta method")
print(asymptotic_cov)

print("Frobenius norm relative error:")
rel_error=np.linalg.norm(draw_cov-asymptotic_cov)/np.linalg.norm(draw_cov)
print(rel_error)

Estimated covariance matrix from draws
[[ 5.27047208e-05 -8.48189139e-05 -4.04843734e-05]
 [-8.48189139e-05  1.40448489e-04  5.88536888e-05]
 [-4.04843734e-05  5.88536888e-05  4.43314061e-05]]
Estimated covariance matrix from delta method
[[ 5.48126131e-05 -8.80335241e-05 -4.24621796e-05]
 [-8.80335241e-05  1.45268781e-04  6.20382061e-05]
 [-4.24621796e-05  6.20382061e-05  4.60040650e-05]]
Frobenius norm relative error:
0.04019034164408226
